#Ecommerce site Recommendation System

---





In this project, we are going to use Collaborative Filtering as well as Item based Filtering to build a recommendation system.

In [160]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from zipfile import ZipFile


We initially obtain a zipped file with the data from the ecommerce site. 
We then create a new file where we write the contents of the zipped file into.
We close the newly created file to reopen it with ZipFile().
We then extract its contents into an extraction path and then close the ZipFile() instance

In [161]:
#url1='https://raw.githubusercontent.com/JuliaKav/Ecommerce-Recommendation-System/main/ecommerce_data.zip'
url='https://github.com/JuliaKav/Ecommerce-Recommendation-System/blob/main/ecommerce_data.zip?raw=true'


In [162]:
zipfile=urlopen(url)

In [163]:
tmpzipfile=open('/tmp/tmpzipfile.zip',mode='wb')
tmpzipfile.write(zipfile.read())
tmpzipfile.close()
zf=ZipFile('/tmp/tmpzipfile.zip')
zf.extractall(path='')
zf.close()

In [164]:
df=pd.read_csv('ecommerce_data.csv', encoding = 'ISO-8859-1')

In [165]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


##Data Analysis

In [166]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [167]:
# We can remove the negative values
df=df.loc[df['Quantity'] >0 ]
df=df.loc[df['UnitPrice']>0]

In [168]:
df.isnull()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
541904,False,False,False,False,False,False,False,False
541905,False,False,False,False,False,False,False,False
541906,False,False,False,False,False,False,False,False
541907,False,False,False,False,False,False,False,False


In [169]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132220
Country             0
dtype: int64

In [170]:
df.shape

(530104, 8)

In [171]:
# We will drop the null values of CustomerID
df=df.dropna(subset=['CustomerID'])

In [172]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

##Customer-Item Matrix
The index of our matrix will be the CustomerID, the columns will be the StockCode for the items and the values displayed will be the Quantity values.

In [173]:
cust_item_mat=df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity' , aggfunc='sum')

In [174]:
cust_item_mat.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16010,16011,16012,16014,16015,16016,16020C,16033,16043,16045,16046,16048,16049,16052,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [175]:
# Replace the NANs with 0's and values by 1

In [176]:
cust_item_mat=cust_item_mat.applymap(lambda x: 1 if x>0 else 0)

In [177]:
cust_item_mat.head()


StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16010,16011,16012,16014,16015,16016,16020C,16033,16043,16045,16046,16048,16049,16052,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


##Collaborative Filtering- User to User Similarity Matrix

In [178]:
from sklearn.metrics.pairwise import cosine_similarity
usr_sim_mat=pd.DataFrame(cosine_similarity(cust_item_mat))

In [179]:
usr_sim_mat


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4298,4299,4300,4301,4302,4303,4304,4305,4306,4307,4308,4309,4310,4311,4312,4313,4314,4315,4316,4317,4318,4319,4320,4321,4322,4323,4324,4325,4326,4327,4328,4329,4330,4331,4332,4333,4334,4335,4336,4337
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117851,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.100504,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,0.060262,0.000000,0.141447,0.057695,0.000000,0.139000,0.061637,0.047108,0.084029,0.000000,0.008240,0.012414,0.084491,0.026334,0.051457,0.079002,0.104510,0.113190,0.031556,0.101060,0.141323,0.053754,0.096619,0.000000,0.013051,0.017418,0.041091,0.086235,0.000000,0.154936,...,0.000000,0.086133,0.017697,0.0,0.059120,0.015204,0.077897,0.044065,0.040226,0.038162,0.150959,0.103557,0.0,0.015778,0.069673,0.064505,0.073900,0.083090,0.000000,0.090041,0.000000,0.095626,0.022605,0.054656,0.051312,0.118835,0.0,0.037242,0.000000,0.049515,0.0,0.029709,0.052668,0.000000,0.032844,0.062318,0.000000,0.113776,0.109364,0.012828
2,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,0.000000,0.059131,0.014574,0.020806,0.067420,0.030076,0.266733,0.152894,0.000000,0.064282,0.017829,0.026861,0.036564,0.056980,0.074227,0.056980,0.025126,0.086441,0.034139,0.024296,0.050965,0.046524,0.041812,0.000000,0.000000,0.037689,0.044455,0.031099,0.041812,0.067049,...,0.000000,0.000000,0.000000,0.0,0.021320,0.032898,0.000000,0.000000,0.000000,0.137620,0.025126,0.034473,0.0,0.034139,0.000000,0.046524,0.266501,0.022473,0.000000,0.000000,0.000000,0.091960,0.000000,0.059131,0.138782,0.000000,0.0,0.000000,0.000000,0.021427,0.0,0.064282,0.113961,0.000000,0.000000,0.000000,0.000000,0.000000,0.170905,0.083269
3,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,0.153389,0.129845,0.136013,0.091376,0.111035,0.140343,0.048810,0.069945,0.024953,0.035289,0.048937,0.029492,0.100362,0.093842,0.081497,0.031281,0.055174,0.071180,0.112449,0.080028,0.097924,0.102162,0.183629,0.111035,0.139522,0.103451,0.024405,0.034144,0.160676,0.098154,...,0.000000,0.045472,0.042042,0.0,0.081929,0.054180,0.018506,0.039257,0.000000,0.090660,0.151728,0.075698,0.0,0.112449,0.000000,0.102162,0.000000,0.098698,0.047782,0.166372,0.022525,0.075725,0.000000,0.064923,0.000000,0.052934,0.0,0.000000,0.035289,0.211735,0.0,0.105868,0.000000,0.000000,0.039014,0.000000,0.000000,0.067574,0.137124,0.030475
4,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,0.021190,0.067267,0.000000,0.071007,0.076696,0.085536,0.000000,0.057977,0.000000,0.073127,0.020282,0.030557,0.083189,0.064820,0.042220,0.064820,0.085749,0.000000,0.077674,0.027639,0.028989,0.052926,0.047565,0.076696,0.000000,0.085749,0.000000,0.070755,0.142695,0.076274,...,0.085749,0.023557,0.000000,0.0,0.097014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026153,0.000000,0.000000,0.031575,0.000000,0.0,0.000000,0.000000,0.024376,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044866,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [180]:
usr_sim_mat.index=cust_item_mat.index

In [181]:
usr_sim_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4298,4299,4300,4301,4302,4303,4304,4305,4306,4307,4308,4309,4310,4311,4312,4313,4314,4315,4316,4317,4318,4319,4320,4321,4322,4323,4324,4325,4326,4327,4328,4329,4330,4331,4332,4333,4334,4335,4336,4337
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117851,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.100504,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,0.060262,0.000000,0.141447,0.057695,0.000000,0.139000,0.061637,0.047108,0.084029,0.000000,0.008240,0.012414,0.084491,0.026334,0.051457,0.079002,0.104510,0.113190,0.031556,0.101060,0.141323,0.053754,0.096619,0.000000,0.013051,0.017418,0.041091,0.086235,0.000000,0.154936,...,0.000000,0.086133,0.017697,0.0,0.059120,0.015204,0.077897,0.044065,0.040226,0.038162,0.150959,0.103557,0.0,0.015778,0.069673,0.064505,0.073900,0.083090,0.000000,0.090041,0.000000,0.095626,0.022605,0.054656,0.051312,0.118835,0.0,0.037242,0.000000,0.049515,0.0,0.029709,0.052668,0.000000,0.032844,0.062318,0.000000,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,0.000000,0.059131,0.014574,0.020806,0.067420,0.030076,0.266733,0.152894,0.000000,0.064282,0.017829,0.026861,0.036564,0.056980,0.074227,0.056980,0.025126,0.086441,0.034139,0.024296,0.050965,0.046524,0.041812,0.000000,0.000000,0.037689,0.044455,0.031099,0.041812,0.067049,...,0.000000,0.000000,0.000000,0.0,0.021320,0.032898,0.000000,0.000000,0.000000,0.137620,0.025126,0.034473,0.0,0.034139,0.000000,0.046524,0.266501,0.022473,0.000000,0.000000,0.000000,0.091960,0.000000,0.059131,0.138782,0.000000,0.0,0.000000,0.000000,0.021427,0.0,0.064282,0.113961,0.000000,0.000000,0.000000,0.000000,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,0.153389,0.129845,0.136013,0.091376,0.111035,0.140343,0.048810,0.069945,0.024953,0.035289,0.048937,0.029492,0.100362,0.093842,0.081497,0.031281,0.055174,0.071180,0.112449,0.080028,0.097924,0.102162,0.183629,0.111035,0.139522,0.103451,0.024405,0.034144,0.160676,0.098154,...,0.000000,0.045472,0.042042,0.0,0.081929,0.054180,0.018506,0.039257,0.000000,0.090660,0.151728,0.075698,0.0,0.112449,0.000000,0.102162,0.000000,0.098698,0.047782,0.166372,0.022525,0.075725,0.000000,0.064923,0.000000,0.052934,0.0,0.000000,0.035289,0.211735,0.0,0.105868,0.000000,0.000000,0.039014,0.000000,0.000000,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,0.021190,0.067267,0.000000,0.071007,0.076696,0.085536,0.000000,0.057977,0.000000,0.073127,0.020282,0.030557,0.083189,0.064820,0.042220,0.064820,0.085749,0.000000,0.077674,0.027639,0.028989,0.052926,0.047565,0.076696,0.000000,0.085749,0.000000,0.070755,0.142695,0.076274,...,0.085749,0.023557,0.000000,0.0,0.097014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026153,0.000000,0.000000,0.031575,0.000000,0.0,0.000000,0.000000,0.024376,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044866,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [182]:
usr_sim_mat.columns=usr_sim_mat.index

In [183]:
usr_sim_mat

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12357.0,12358.0,12359.0,12360.0,12361.0,12362.0,12363.0,12364.0,12365.0,12367.0,12370.0,12371.0,12372.0,12373.0,12374.0,12375.0,12377.0,12378.0,12379.0,12380.0,12381.0,12383.0,12384.0,12386.0,12388.0,12390.0,12391.0,12393.0,12394.0,12395.0,...,18230.0,18231.0,18232.0,18233.0,18235.0,18236.0,18237.0,18239.0,18240.0,18241.0,18242.0,18245.0,18246.0,18248.0,18249.0,18250.0,18251.0,18252.0,18255.0,18257.0,18259.0,18260.0,18261.0,18262.0,18263.0,18265.0,18268.0,18269.0,18270.0,18272.0,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117851,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.100504,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,0.060262,0.000000,0.141447,0.057695,0.000000,0.139000,0.061637,0.047108,0.084029,0.000000,0.008240,0.012414,0.084491,0.026334,0.051457,0.079002,0.104510,0.113190,0.031556,0.101060,0.141323,0.053754,0.096619,0.000000,0.013051,0.017418,0.041091,0.086235,0.000000,0.154936,...,0.000000,0.086133,0.017697,0.0,0.059120,0.015204,0.077897,0.044065,0.040226,0.038162,0.150959,0.103557,0.0,0.015778,0.069673,0.064505,0.073900,0.083090,0.000000,0.090041,0.000000,0.095626,0.022605,0.054656,0.051312,0.118835,0.0,0.037242,0.000000,0.049515,0.0,0.029709,0.052668,0.000000,0.032844,0.062318,0.000000,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,0.000000,0.059131,0.014574,0.020806,0.067420,0.030076,0.266733,0.152894,0.000000,0.064282,0.017829,0.026861,0.036564,0.056980,0.074227,0.056980,0.025126,0.086441,0.034139,0.024296,0.050965,0.046524,0.041812,0.000000,0.000000,0.037689,0.044455,0.031099,0.041812,0.067049,...,0.000000,0.000000,0.000000,0.0,0.021320,0.032898,0.000000,0.000000,0.000000,0.137620,0.025126,0.034473,0.0,0.034139,0.000000,0.046524,0.266501,0.022473,0.000000,0.000000,0.000000,0.091960,0.000000,0.059131,0.138782,0.000000,0.0,0.000000,0.000000,0.021427,0.0,0.064282,0.113961,0.000000,0.000000,0.000000,0.000000,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,0.153389,0.129845,0.136013,0.091376,0.111035,0.140343,0.048810,0.069945,0.024953,0.035289,0.048937,0.029492,0.100362,0.093842,0.081497,0.031281,0.055174,0.071180,0.112449,0.080028,0.097924,0.102162,0.183629,0.111035,0.139522,0.103451,0.024405,0.034144,0.160676,0.098154,...,0.000000,0.045472,0.042042,0.0,0.081929,0.054180,0.018506,0.039257,0.000000,0.090660,0.151728,0.075698,0.0,0.112449,0.000000,0.102162,0.000000,0.098698,0.047782,0.166372,0.022525,0.075725,0.000000,0.064923,0.000000,0.052934,0.0,0.000000,0.035289,0.211735,0.0,0.105868,0.000000,0.000000,0.039014,0.000000,0.000000,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,0.021190,0.067267,0.000000,0.071007,0.076696,0.085536,0.000000,0.057977,0.000000,0.073127,0.020282,0.030557,0.083189,0.064820,0.042220,0.064820,0.085749,0.000000,0.077674,0.027639,0.028989,0.052926,0.047565,0.076696,0.000000,0.085749,0.000000,0.070755,0.142695,0.076274,...,0.085749,0.023557,0.000000,0.0,0.097014,0.000000,0.000000,0.000000,0.000000,0.00000

We will now look to find which customer is most similar to the customer with id 12358.

In [184]:
usr_sim_mat.loc[12358].sort_values(ascending =False)

CustomerID
12358.0    1.000000
14145.0    0.452911
14155.0    0.452911
18240.0    0.452911
13551.0    0.416025
             ...   
16099.0    0.000000
16098.0    0.000000
16097.0    0.000000
16096.0    0.000000
12346.0    0.000000
Name: 12358.0, Length: 4338, dtype: float64

What items did customer 12358 buy?

In [185]:
cust_item_mat.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16010,16011,16012,16014,16015,16016,16020C,16033,16043,16045,16046,16048,16049,16052,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [186]:
items_of_12358=cust_item_mat.loc[12358]
items_of_12358

StockCode
10002     0
10080     0
10120     0
10123C    0
10124A    0
         ..
C2        0
DOT       0
M         0
PADS      0
POST      1
Name: 12358.0, Length: 3665, dtype: int64

In [187]:
#items_cust_12358=set(cust_item_mat.loc[12358].iloc[cust_item_mat.loc[12358].to_numpy().nonzero()] .index) 
items_of_12358= set(items_of_12358.iloc[items_of_12358.to_numpy().nonzero()].index)
items_of_12358

{'15056BL',
 '15056N',
 '15056P',
 '15060B',
 '20679',
 '21232',
 '22059',
 '22063',
 '22646',
 '37447',
 '37449',
 '48185',
 'POST'}

In [188]:
# Similarily for item 14145

In [189]:
items_of_14145=cust_item_mat.loc[14145] #set(cust_item_mat.loc[14145].iloc[cust_item_mat.loc[14145].to_numpy().nonzero()] .index) 

In [190]:
items_of_14145=set(items_of_14145.iloc[items_of_14145.to_numpy().nonzero()].index)
items_of_14145

{'15056BL', '15056N', '15056P', '20679', '85014A', '85014B'}

In [191]:
# Obtain the items bought by 12358 but not 14145

In [192]:
recomm_items=items_of_12358 - items_of_14145

In [193]:
recomm_items

{'15060B',
 '21232',
 '22059',
 '22063',
 '22646',
 '37447',
 '37449',
 '48185',
 'POST'}

In [194]:
# We can recommend these items to 14145. We now need their descriptions

In [195]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [196]:
df.loc[df['StockCode'].isin(recomm_items),['StockCode','Description']].set_index('StockCode')

,Description
StockCode,
POST,POSTAGE
22646,CERAMIC STRAWBERRY CAKE MONEY BANK
48185,DOORMAT FAIRY CAKE
21232,STRAWBERRY CERAMIC TRINKET BOX
POST,POSTAGE
...,...
22059,CERAMIC STRAWBERRY DESIGN MUG
22646,CERAMIC STRAWBERRY CAKE MONEY BANK
POST,POSTAGE


##Recommending items to a customer

In [202]:
# For a given customer, we first need to find another customer with the most similarities. For this we use a user similarity matrix
# Once we found the most similar customer, we obtain a list of items bought by this customer.
# We compare items bought by both customers and see which ones were bought by the second customer that were not bought by the first.
# We return this list of items to be recommended to the first customer.

In [252]:
def get_cust_items(custid):
  cust_items=cust_item_mat.loc[custid]
  cust_items=set(cust_items.iloc[cust_items.to_numpy().nonzero()].index)
  return cust_items

In [253]:
def get_recommended_items(custid):
  cust_items=get_cust_items(custid)
  similar_cust=usr_sim_mat.loc[custid].sort_values(ascending=False).reset_index().iloc[1,0]
  custb_items=get_cust_items(similar_cust)
  diff_items=custb_items - cust_items
  recomm_items=df.loc[df['StockCode'].isin(diff_items),['StockCode','Description']].set_index('StockCode')
  return recomm_items


In [254]:
recommended_items=get_recommended_items(12358)
recommended_items

,Description
StockCode,
85014B,RED RETROSPOT UMBRELLA
85014A,BLACK/BLUE POLKADOT UMBRELLA
85014B,RED RETROSPOT UMBRELLA
85014A,BLACK/BLUE POLKADOT UMBRELLA
85014A,BLACK/BLUE POLKADOT UMBRELLA
...,...
85014B,RED RETROSPOT UMBRELLA
85014A,BLACK/BLUE POLKADOT UMBRELLA
85014B,RED RETROSPOT UMBRELLA


##Using Item based Collaborative Filtering to obtain item similarities

In [255]:
transp_cust_item_mat=cust_item_mat.transpose()
transp_cust_item_mat

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12357.0,12358.0,12359.0,12360.0,12361.0,12362.0,12363.0,12364.0,12365.0,12367.0,12370.0,12371.0,12372.0,12373.0,12374.0,12375.0,12377.0,12378.0,12379.0,12380.0,12381.0,12383.0,12384.0,12386.0,12388.0,12390.0,12391.0,12393.0,12394.0,12395.0,...,18230.0,18231.0,18232.0,18233.0,18235.0,18236.0,18237.0,18239.0,18240.0,18241.0,18242.0,18245.0,18246.0,18248.0,18249.0,18250.0,18251.0,18252.0,18255.0,18257.0,18259.0,18260.0,18261.0,18262.0,18263.0,18265.0,18268.0,18269.0,18270.0,18272.0,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10123C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10124A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DOT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
M,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [256]:
it_mat=pd.DataFrame(cosine_similarity(transp_cust_item_mat))

In [257]:
it_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3625,3626,3627,3628,3629,3630,3631,3632,3633,3634,3635,3636,3637,3638,3639,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649,3650,3651,3652,3653,3654,3655,3656,3657,3658,3659,3660,3661,3662,3663,3664
0,1.000000,0.000000,0.094868,0.091287,0.0,0.000000,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,0.062284,0.043437,0.079057,0.072336,0.152030,0.053074,0.069938,0.033903,0.025000,0.055902,0.031009,0.076696,0.043033,0.182574,0.076948,0.027524,0.054233,0.036274,0.049386,0.158114,0.0,0.0,0.040489,0.043853,0.024693,0.000000,0.070711,...,0.0,0.055902,0.0,0.0,0.000000,0.0,0.091287,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.070711,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.029361,0.0,0.067591,0.0,0.078217
1,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,0.030124,0.000000,0.000000,0.000000,0.031513,0.000000,0.050738,0.049192,0.000000,0.000000,0.000000,0.027821,0.062439,0.000000,0.037216,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.117495,0.000000,0.035829,0.102598,0.000000,...,0.0,0.081111,0.0,0.0,0.102598,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.162221,0.102598,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.016345,0.0,0.000000
2,0.094868,0.000000,1.000000,0.115470,0.0,0.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,0.026261,0.000000,0.000000,0.000000,0.027472,0.067135,0.029488,0.021442,0.031623,0.035355,0.039223,0.024254,0.027217,0.115470,0.032444,0.139262,0.068599,0.045883,0.031235,0.000000,0.0,0.0,0.076822,0.110940,0.031235,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.115470,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.089443,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.071247,0.0,0.010993
3,0.091287,0.000000,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,1.0,0.447214,0.063888,0.044499,0.000000,0.000000,0.000000,0.000000,0.064216,0.058722,0.061430,0.000000,0.000000,0.061430,0.037529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072548,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.124035,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3660,0.029361,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.036955,0.019360,0.055989,0.000000,0.000000,0.039996,0.000000,0

In [258]:
it_mat.columns=transp_cust_item_mat.index

In [259]:
it_mat['StockCode']=transp_cust_item_mat.index

In [260]:
it_mat=it_mat.set_index('StockCode')

In [261]:
it_mat

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16010,16011,16012,16014,16015,16016,16020C,16033,16043,16045,16046,16048,16049,16052,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.091287,0.0,0.000000,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,0.062284,0.043437,0.079057,0.072336,0.152030,0.053074,0.069938,0.033903,0.025000,0.055902,0.031009,0.076696,0.043033,0.182574,0.076948,0.027524,0.054233,0.036274,0.049386,0.158114,0.0,0.0,0.040489,0.043853,0.024693,0.000000,0.070711,...,0.0,0.055902,0.0,0.0,0.000000,0.0,0.091287,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.070711,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.029361,0.0,0.067591,0.0,0.078217
10080,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,0.030124,0.000000,0.000000,0.000000,0.031513,0.000000,0.050738,0.049192,0.000000,0.000000,0.000000,0.027821,0.062439,0.000000,0.037216,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.117495,0.000000,0.035829,0.102598,0.000000,...,0.0,0.081111,0.0,0.0,0.102598,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.162221,0.102598,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.016345,0.0,0.000000
10120,0.094868,0.000000,1.000000,0.115470,0.0,0.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,0.026261,0.000000,0.000000,0.000000,0.027472,0.067135,0.029488,0.021442,0.031623,0.035355,0.039223,0.024254,0.027217,0.115470,0.032444,0.139262,0.068599,0.045883,0.031235,0.000000,0.0,0.0,0.076822,0.110940,0.031235,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.115470,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.089443,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.071247,0.0,0.010993
10123C,0.091287,0.000000,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.000000,0.000000,0.000000,1.0,0.447214,0.063888,0.044499,0.000000,0.000000,0.000000,0.000000,0.064216,0.058722,0.061430,0.000000,0.000000,0.061430,0.037529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072548,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.124035,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [268]:
#Most similar item to 10080
most_sim_item=it_mat.loc['10080'].sort_values(ascending=False).reset_index().iloc[1,0]


'23694'

In [272]:
#10 most similar items
most_sim_10=it_mat.loc['10080'].sort_values(ascending=False).reset_index().iloc[1:11,0]

In [273]:
most_sim_10

1      23694
2      22039
3     47504H
4      21650
5     90214F
6     79157B
7     90206A
8      84012
9      22043
10     21373
Name: StockCode, dtype: object

In [277]:
df.loc[df['StockCode'].isin(most_sim_10),['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
47504H,ENGLISH ROSE SPIRIT LEVEL
22043,CHRISTMAS CARD SCREEN PRINT
21650,ASSORTED TUTTI FRUTTI BRACELET
84012,MAGIC SHEEP WOOL GROWING FROM PAPER
22039,BOTANICAL LILY GIFT WRAP
90206A,GOLD DIAMANTE STAR BROOCH
21373,MIRRORED WALL ART SNOWFLAKES
79157B,UBO-LIGHT TRIOBASE BLUE
23694,PAISLEY PARK CARD


In [278]:
#Function to get 10 most similar items for specified input

In [284]:
def get_10_most_sim_items(itemid):
  most_sim_10=it_mat.loc[itemid].sort_values(ascending=False).reset_index().iloc[1:11,0]
  items=df.loc[df['StockCode'].isin(most_sim_10),['StockCode', 'Description']].drop_duplicates().set_index('StockCode').loc[most_sim_10]
  return items

In [285]:
get_10_most_sim_items('84029E')

,Description
StockCode,
84029G,KNITTED UNION FLAG HOT WATER BOTTLE
21479,WHITE SKULL HOT WATER BOTTLE
21485,RETROSPOT HEART HOT WATER BOTTLE
22111,SCOTTIE DOG HOT WATER BOTTLE
22112,CHOCOLATE HOT WATER BOTTLE
22114,HOT WATER BOTTLE TEA AND SYMPATHY
23355,HOT WATER BOTTLE KEEP CALM
84030E,ENGLISH ROSE HOT WATER BOTTLE
22632,HAND WARMER RED POLKA DOT
